In [ ]:
import numpy as np
import sys
import os
import subprocess
import time
import hashlib

LTSPICE_EXE= "..\\..\\AppData\\Local\\Programs\\ADI\\LTspice\\LTspice.exe"
CIRCUIT_ASC_NAME = ".\\SimFiles\\MainCircuit"
THERMAL_ASC_NAME = ".\\SimFiles\\ThermalApproxRC"
DATA_SAVES_PATH  = ".\\SimFiles\\Saves\\Interpolation\\"    # location to save sim files

LOG_FILE_HANDLE = open(DATA_SAVES_PATH + "BATCH_LOG.txt", "w+")


def FindOperatingPointFrequency(Cs, Ls, Lp, R, G):
    """
        Returns the frequency in Hertz for a given operating point
        
        ### Parameters:
            - Cs = Series Capacitor
            - Ls = Series Inductor
            - Lp = Parllel Inductor
            - R  = LLC Equivalent Resistor representing the output load resistance after transformer gain and recification
            - G  = The DC/DC Gain for the whole converter   

        ### Return:
            - Frequency in Hz
    """

    Coef_A = (Ls/R)**2
    Coef_B = (Ls/Lp)**2 + 2*(Ls/Lp) + 1 - 1/G**2 - 2*Ls/(Cs * R**2)
    Coef_C = 1/(Cs * R)**2 - 2*Ls/(Cs * Lp**2) - 2/(Cs * Lp)
    Coef_D = 1/((Cs*Lp)**2)

    gain_eqn = [Coef_A, 0, Coef_B, 0, Coef_C, 0, Coef_D]
    roots = np.roots(gain_eqn)

    real_solutions = []
    for r in roots:
        if np.imag(r) == 0:
            real_solutions.append(np.real(r))

    if len(real_solutions) == 0:
        return -1
    else:
        return round(max(real_solutions) / (2 * np.pi))

def FindInitialConditions(Cs, Ls, Lp, R, F, Vin):
    """
        Returns a vector of initial conditions of the LLC Tank and DC Link Capacitors relative to the start of the LLC chopped AC Voltage Input
        
        ### Parameters:
            - Cs = Series Capacitor
            - Ls = Series Inductor
            - Lp = Parllel Inductor
            - R  = LLC Equivalent Resistor representing the output load resistance after transformer gain and recification
            - F  = Switching Frequency in Hz
            - Vin = Input DC Voltage to the Converter 

        ### Returns:
            - list [Vdc V_LLC_In V_LLC_Out Vcs Vls Is Ip]
            1. DC Link Voltage Top
            2. DC Link Voltage Bottom
            3. LLC Tank Input Voltage (Square wave with amplitude half converter input voltage)
            4. LLC Tank Output Voltage (over equivalent resistor)
            5. Voltage over the series capacitor
            6. Voltage over the series inductor
            7. Series Current
            8. Parallel Current through parallel inductor
    """

    w = 2 * np.pi * F

    Xlp = 1j * w * Lp
    Xls = 1j * w * Ls
    Xc = 1 / (1j * w * Cs)

    Z1 = Xc + Xls
    Z2 = (Xlp * R) / (Xlp + R)
    Vac = Vin/2 * 4/np.pi

    Is = Vac / (Z1 + Z2)
    Vcs = Is * Xc
    Vls = Is * Xls
    Vout = Vac * (Z2 / (Z1 + Z2))
    Ip = Vout / Xlp

    # We need to get the initial conditions of each parameter at a specific phase
    # The phase is relative to the LLC Input chopped voltage at the start of its cycle
    ic = lambda x : np.abs(x) * np.sin(-np.angle(x))
    return [Vin/2, Vin/2, Vin/2, ic(Vout), ic(Vcs), ic(Vls), ic(Is), ic(Ip)]

def EngNotation(value):
    """
        Returns a string of the number with a metrix prefix at the end. It has a range from pico to tera in terms of the prefix.
        Anything below a femto (0.001p) will result in zero.
    """
    if(value == 0):
        return ("0")
    
    comLog = np.log10(abs(value))
    metricScale = int( np.floor( comLog / 3 ) )
    power = metricScale * 3
    prefix = ['p', 'n', 'u', 'm', ' ', 'K', 'Meg', 'G', 'T']

    if metricScale < -4:
        metricScale = -4
        power = metricScale * 3
        mantissa = value / (10 ** power)
    elif metricScale > 4:
        metricScale = 4
        power = metricScale * 3
        mantissa = value / (10 ** power)
    else:
        mantissa = value / (10 ** power)
        
    return ("%0.3f%s" % (mantissa, prefix[metricScale + 4]))

def SimFailCheck(CIRCUIT_NAME):
    """
        Checks the simulation log file for a FATAL ERROR, measurement FAIL'ed, or a generic Error message. This will return a success flag
        and the log file failure line if a failure exists
    """
    log_file = open(CIRCUIT_NAME + ".log", 'rb')
    log_line = log_file.readline().replace(b'\x00', b'') # For some reason log files are normally utf-8, but if there is a failure due to
                                                         # missing parameters, the file format changes to utf-16 but still has trouble
                                                         # decoding with python. So we just remove the leading 8bits which are basically all 0
                                                         # And force a conversion down to utf-8 to read the file as normal.

    while( (not log_line.startswith(b"Fatal Error")) and (not log_line.rstrip().endswith(b"FAIL\'ed")) and log_line != b""):
        log_line = log_file.readline().replace(b'\x00', b'')
    log_file.close()
    
    if log_line == b"":
        return [True, ""]
    else:
        return [False, log_line.decode('utf-8')]

def TitleParamHash(paramDirectiveString):
    """
        This fuction returns a file title for a given simulation operating point. 
        It produces the title by hashing a string of the .param directive 
    """
    hashTitle = hashlib.md5(paramDirectiveString.encode('utf-8'))
    return hashTitle.hexdigest() 

def CreateMeasurementsString(Cdc, Cs, Lp, Ls, Cds_Slew, Cgs_Slew, L_gate, L_drain, F):
    '''
        Dynamically creates the measurement directives. Assumes measuements of three cycles ignoring the first to reach steady state
    '''

    meas_type = ["AVG" ,"MIN", "MAX", "PP", "RMS"]
    
    names = ["I_SER", "I_SER_SLEW", "I_PARA", "I_PARA_SLEW", "I_OUT", "V_CS", "V_CS_SLEW",
                "V_LS", "V_OUT", "P_OUT", "I_IN", "P_IN"]
    pin_name = ["I(Cs)", "V(A,Vo)/" + EngNotation(Ls), "I(Lp)", "V(Vo,Vn)/" + EngNotation(Lp), 
                "I(Rac)", "V(Vp,A)", "I(Cs)/" + EngNotation(Cs), "V(A,Vo)", "V(Vo,Vn)", "V(Vo,Vn)*I(Rac)", 
                "I(Vin)", "V(Vdc)*I(Vin)"]

    names_switch = ["V_CDC", "V_CDC_SLEW", "I_CDC", "V_DS", "V_DS_SLEW", "I_D", "I_D_SLEW", "P_SW", 
                    "V_GS", "V_GS_SLEW", "V_RG", "I_G", "I_G_SLEW", "Q_GATE"]
    

    pin_name_switch_top = ["V(Vdc,vn)", "I(C1)/" + EngNotation(Cdc), "I(C1)", "V(Vd1,Vp)", 
                           "I(C_vds1_slew)/" + EngNotation(Cds_Slew), "I(Id_meas1)", "V(Vd1,x1:vd_fet)/" + EngNotation(L_drain),
                            "V(Vd1,Vp)*I(Id_meas1)", "V(Vg1,Vp)", "I(C_vgs1_slew)/" + EngNotation(Cgs_Slew),"V(G1,Vg1)",
                         "I(Rg1)", "V(vg1,x1:vg_mos)/" + EngNotation(L_gate), "I(Q_g1)", "ERROR_ZVS_TODO"]
    
    pin_name_switch_bot = ["V(Vn)", "I(C2)/" + EngNotation(Cdc), "I(C2)", "V(Vd2,Vss)", 
                           "I(C_vds2_slew)/" + EngNotation(Cds_Slew), "I(Id_meas2)", "V(Vd2,x2:vd_fet)/" + EngNotation(L_drain),
                            "V(Vd2,Vss)*I(Id_meas2)", "V(Vg2,Vss)", "I(C_vgs2_slew)/" + EngNotation(Cgs_Slew),"V(G2,Vg2)",
                         "I(Rg2)", "V(vg2,x2:vg_mos)/" + EngNotation(L_gate), "I(Q_g2)"]


    directives = []
    START = EngNotation(1/F)
    END = EngNotation(4/F)

    for i in range(len(names)):
        for measType  in meas_type:
            currName = names[i] + "_" + measType
            directives.append(".meas TRAN %s %s %s FROM %s TO %s\n" % (currName, measType, pin_name[i], START, END))

    for i in range(len(names_switch)):
        for measType in meas_type:
            currNameTop = names_switch[i] + "_" + measType + "_T"
            currNameBot = names_switch[i] + "_" + measType + "_B"
            directives.append(".meas TRAN %s %s %s FROM %s TO %s\n" % (currNameTop, measType, pin_name_switch_top[i], START, END))
            directives.append(".meas TRAN %s %s %s FROM %s TO %s\n" % (currNameBot, measType, pin_name_switch_bot[i], START, END))

    directives.append(".meas TRAN ZVS_T TRIG I(ZVS_Sig1)=0 TD=%s FALL=1 TARG V(G1,Vp)=0 TD=%s RISE=1\n" % (START, START))
    directives.append(".meas TRAN ZVS_B TRIG I(ZVS_Sig2)=0 TD=%s FALL=1 TARG V(G2,Vss)=0 TD=%s RISE=1\n" % (EngNotation(1.5/F), EngNotation(1.5/F)))

    return directives


'''
=====================================================================================
                            Batch Setup Inputs
=====================================================================================
'''

SIM_TIMEOUT_SEC = 60 * 10
THERM_TIMEOUT_SEC = 60 * 2.5

MAX_SIM_TIME_MINS = 2.7 * (24 * 60) # Represents a day of sim time to check in on
AVG_SIM_TIME_MINS = 2.3


#Parameters and their ranges here

NUM_PARAMS = 16
PARAM_NAMES_LIST = ["Cs", "Ls", "Lp", "P", "Vin", "L_Loop", "ESR_Cs", "ESR_Ls", "ESR_Lp", "ESR_Cdc", "Rds_on_Top", "Rds_on_Bottom", "Cds_Top", "Cds_Bottom", "Cgs_Top", "Cds_Bottom"]
PARAM_RANGE_LIST = [
    [239e-9 * (0.5), 239e-9 * (1.5)],
    [10.58e-6 * (0.5), 10.58e-6 * (1.5)],
    [105.8e-6 * (0.5), 105.8e-6 * (1.5)],
    [1e3, 39e3],
    [600, 1000],
    [0.5e-9, 20e-9],
    [20e-3*0.75, 20e-3*1.25],
    [10e-3, 75e-3],
    [10e-3, 75e-3],
    [10e-3*0.8, 10e-3*1.2],
    [0, 50e-3],
    [0, 50e-3],
    [0, 350e-12*0.3],
    [0, 350e-12*0.3],
    [0, 8500e-12*0.3],
    [0, 8500e-12*0.3]
]

CS_INDEX = 0
LS_INDEX = 1
LP_INDEX = 2
P_INDEX = 3
VIN_INDEX = 4
L_LOOP_INDEX = 5
ESR_CS_INDEX = 6
ESR_LS_INDEX = 7
ESR_LP_INDEX = 8
ESR_CDC_INDEX = 9
RDS_ON_INDEX = 10
CDS_INDEX = 11
CGS_INDEX = 12

C_DC = 1e-3
C_DS_SLEW = 0.035e-12 
C_GS_SLEW = 0.85e-12
L_GATE = 10e-9
L_DRAIN = 2e-9


SIM_POOL_SIZE = int(MAX_SIM_TIME_MINS / AVG_SIM_TIME_MINS)

In [ ]:
# Loop Over the Sim ID's (SID)
BATCH_START_TIME = time.time()

SID_I = -1
while SID_I < (SIM_POOL_SIZE - 1):
    SID_I += 1

    ''' USED FOR MIN MAX SIMS
    # For the 16 parameters, we choose a random 16 bit number where 0 gives the min for that param and 1 gives the max
    # Randomly select for better coverage, save SID in file to quickly skip repeat sims
    SID = np.random.randint(0, 2**16 - 1)
    SID_BIN = format(SID, "016b")

    print("PROGRESS (%d): %d/%d" % (SID, SID_I, SIM_POOL_SIZE - 1)) 
    
    # convert SID to parameter list
    #paramStepIndex = SIDToParams(SID, PARAM_STEP_COUNT_LIST)
    # Use SID to get parameter values 
    paramsOP = []
    #paramsOP = [239e-9, 10.58e-6, 105.8e-6, 1, 800, 1e-9, 20e-3, 10e-3, 10e-3, 60e-3, 0, 0, 0, 0, 0, 0]
    
    # Turn random number into operating point
    for i in range(len(PARAM_NAMES_LIST)):
        MinMax = int(SID_BIN[i])
        value = PARAM_RANGE_LIST[i][MinMax]
        paramsOP.append(value)
    '''

    '''USED FOR INTERPOLATION DATA SIMS'''
    subDivision = 25

    ranges = np.array(PARAM_RANGE_LIST)
    lowerRanges = ranges[:,0]
    upperRanges = ranges[:,1]
    del ranges
    paramsOP = (upperRanges - lowerRanges) * (np.random.randint(1, subDivision - 1, (16,))/subDivision) + lowerRanges
    print("PROGRESS: %d/%d : " % (SID_I, SIM_POOL_SIZE - 1), end="") 


    # Configuration Setup
    Cs = paramsOP[CS_INDEX]
    Ls = paramsOP[LS_INDEX]
    Lp = paramsOP[LP_INDEX]
    P = paramsOP[P_INDEX]
    Vin = paramsOP[VIN_INDEX]
    L_Loop = paramsOP[L_LOOP_INDEX]
    ESR_Cs = paramsOP[ESR_CS_INDEX]
    ESR_Ls = paramsOP[ESR_LS_INDEX]
    ESR_Lp = paramsOP[ESR_LP_INDEX]
    ESR_Cdc = paramsOP[ESR_CDC_INDEX]
    Rdson_Top = paramsOP[10]
    Rdson_Bottom = paramsOP[11]
    Cds_Top = paramsOP[12]
    Cds_Bottom = paramsOP[13]
    Cgs_Top = paramsOP[14]
    Cgs_Bottom = paramsOP[15]

    Rac = ((Vin ** 2) / P) * 8 / (np.pi ** 2)
    Turns_Ratio = (1/2)
    R = Rac * (Turns_Ratio ** 2)

    # find the operating frequency and initial conditions
    F = FindOperatingPointFrequency(Cs, Ls, Lp, R, 1)
    IC = FindInitialConditions(Cs, Ls, Lp, R, F, Vin)

    paramDirectiveString = ".param Cs=%s Ls=%s Lp=%s Rac=%s Vin=%s F=%s L_Loop=%s Cs_ESR=%s Ls_ESR=%s Lp_ESR=%s Cdc_ESR=%s Rdson_T=%s Rdson_B=%s Cds_T=%s Cds_B=%s Cgs_T=%s Cgs_B=%s\r\n" \
        % (EngNotation(Cs), EngNotation(Ls), EngNotation(Lp), EngNotation(R), EngNotation(Vin), EngNotation(F), EngNotation(L_Loop), \
           EngNotation(ESR_Cs), EngNotation(ESR_Ls), EngNotation(ESR_Lp), EngNotation(ESR_Cdc), EngNotation(Rdson_Top), EngNotation(Rdson_Bottom), \
            EngNotation(Cds_Top), EngNotation(Cds_Bottom), EngNotation(Cgs_Top), EngNotation(Cgs_Bottom))
    initialConditionStrings= ".ic V(Vdc, Vn)=%s V(Vn)=%s V(Vp,Vn)=%s V(Vo,Vn)=%s V(Vp,A)=%s V(A,Vo)=%s I(Ls)=%s I(Lp)=%s\r\n" % tuple([EngNotation(x) for x in IC])
    
    hashTitle = TitleParamHash(paramDirectiveString)
    print(hashTitle)

    # Check if hash already exists to avoid duplicate sims
    dirList = os.listdir(DATA_SAVES_PATH)
    hash_search = len([x for x in dirList if (x.split('.')[0].split('_')[1] == hashTitle)]) > 0
    if(hash_search):
        SID_I -= 1
        print("DUPE FOUND (%d)" % SID_I)
        continue

    
    LOG_FILE_HANDLE.write("STARTING %s\n" % hashTitle)
    LOG_FILE_HANDLE.write("%s%s" % (paramDirectiveString, initialConditionStrings))
    if F == -1:
        LOG_FILE_HANDLE.write("%s FAILED :: COULDN\'T FIND AN OPERATING FREQUENCY\n" % hashTitle)
        continue
        
    
    '''
    ===================Write everything to net file============================
        Produce Netlist, open the file, go to the end and insert commands
    '''
    subprocess.run([LTSPICE_EXE, "-netlist", CIRCUIT_ASC_NAME + ".asc"], shell=True)
    netlist_file = open(CIRCUIT_ASC_NAME + ".net", 'rb+')
    netlist_file.seek(-17,2) #move just above the .end and .backanno lines at the end of the netlist

    netlist_file.write(bytes(paramDirectiveString.encode("utf-8")))
    netlist_file.write(bytes(initialConditionStrings.encode("utf-8")))
    netlist_file.write(b".tran %s\r\n" % EngNotation(4.5/F + 0.1e-6).encode('utf-8'))
    
    # Writing out custom measurements to netlist
    directives = CreateMeasurementsString(C_DC, Cs, Lp, Ls, C_DS_SLEW, C_GS_SLEW, L_GATE, L_DRAIN, F)
    for x in directives:
        netlist_file.write(bytes(x.encode('utf-8')))

    #Add back ending commands and close file
    netlist_file.write(b".backanno\r\n.end\r\n")
    netlist_file.close()
    

    # Run the simulation and timeout/kill the process after set amount
    try:
        subproc = subprocess.Popen([LTSPICE_EXE, "-Run", "-b", CIRCUIT_ASC_NAME + ".net"])
        subproc.wait(timeout = SIM_TIMEOUT_SEC)
    except subprocess.TimeoutExpired:
        subproc.kill()
        print("%s TIMED OUT" % hashTitle)
        LOG_FILE_HANDLE.write("%s TIMED OUT\n" % hashTitle)
        time.sleep(1) # Needed to give some time for LTSpice to close out before trying to rename log/net files otherwise LTSpice has write permissions only
        
    
    '''
    ===================Detect sim/measurement success============================
    '''
    [successFlag, msg] = SimFailCheck(CIRCUIT_ASC_NAME)
    if(successFlag):
        print("%s SUCCESS" % (hashTitle))
        LOG_FILE_HANDLE.write("%s SUCCESS\n" % (hashTitle))
    else:
        print("%s FAILED :: %s\n" % (hashTitle, msg))
        LOG_FILE_HANDLE.write("%s FAILED :: %s\n" % (hashTitle, msg))


    '''
    ===================THERMAL APPROX INIT============================
    '''
    # Find Avg Power Loss in switches
    log_file = open(CIRCUIT_ASC_NAME + ".log", 'rb+')
    line = log_file.readline().translate(None, b'\x00').split(b':')
    switch_avg_losses = [-1, -1]

    while(line[0] != b"" and successFlag):
        if (line[0].strip() == b"p_sw_avg_t"):
            switch_avg_losses[0] = float(line[1].split(b'=')[1].split(b'FROM')[0].strip())
        elif (line[0].strip() == b"p_sw_avg_b"):    
            switch_avg_losses[1] = float(line[1].split(b'=')[1].split(b'FROM')[0].strip())

        line = log_file.readline().translate(None, b'\x00').split(b':')
    log_file.close()

    if (switch_avg_losses[0] <= 0 or switch_avg_losses[1] <= 0) and successFlag:
        print("%s FAILED :: COULDN\'T FIND SWITCH POWER LOSSES\n" % hashTitle)
        LOG_FILE_HANDLE.write("%s FAILED :: COULDN\'T FIND SWITCH POWER LOSSES\n" % hashTitle)
        successFlag = False

    '''
    ===================SAVING FILES============================
        Save all files, rename, and move to a saves folder
    '''
    dirList = os.listdir(".\\SimFiles")
    for file in dirList:
        tokens = file.split('.', 1)
        if tokens[0] == "MainCircuit" and tokens[1] != "asc" and successFlag:
            os.rename(CIRCUIT_ASC_NAME + "." + tokens[1], DATA_SAVES_PATH + "SIM_" + hashTitle + "." + tokens[1])
        elif tokens[0] == "MainCircuit" and tokens[1] != "asc" and not successFlag:
            os.rename(CIRCUIT_ASC_NAME + "." + tokens[1], DATA_SAVES_PATH + "SIM_" + hashTitle + "_FAILED." + tokens[1])

    # Skip thermal sims if simulation broke
    if(not successFlag):
        continue

    '''
    ===================THERMAL APPROX SIM============================
    '''
    LOG_FILE_HANDLE.write("STARTING THERMAL SIMS: P_T=%f P_B=%f\n" % (switch_avg_losses[0], switch_avg_losses[1]))
    for i in range(2):
        subprocess.run([LTSPICE_EXE, "-netlist", THERMAL_ASC_NAME + ".asc"], shell=True)
        netlist_file = open(THERMAL_ASC_NAME + ".net", 'rb+')
        netlist_file.seek(-17,2) #move just above the .end and .backanno lines at the end of the netlist
        powerParam = ".param Pavg_Loss = %s\r\n" % str(switch_avg_losses[i])
        netlist_file.write(bytes(powerParam.encode('utf-8')))
        netlist_file.write(b".backanno\r\n.end\r\n")
        netlist_file.close()

        try:
            subproc = subprocess.Popen([LTSPICE_EXE, "-Run", "-b",THERMAL_ASC_NAME + ".net"])
            subproc.wait(timeout = THERM_TIMEOUT_SEC)
            
            if i == 0:
                time.sleep(5)
                os.rename(THERMAL_ASC_NAME + ".log", DATA_SAVES_PATH + "THERM_T_" + hashTitle + ".log")
            else:
                time.sleep(5)
                os.rename(THERMAL_ASC_NAME + ".log", DATA_SAVES_PATH + "THERM_B_" + hashTitle + ".log")
                os.remove(THERMAL_ASC_NAME + ".net")
                os.remove(THERMAL_ASC_NAME + ".op.raw")
                os.remove(THERMAL_ASC_NAME + ".raw")
                
        except subprocess.TimeoutExpired: 
            subproc.kill()
            print("%s TIMED OUT THERMAL" % hashTitle)
            LOG_FILE_HANDLE.write("%s THERMAL TIMED OUT\n" % hashTitle)
            time.sleep(1)
            
        

    
BATCH_END_TIME = time.time()
LOG_FILE_HANDLE.write("\nBATCH SIMS COMPLETED in %ss" % str(BATCH_END_TIME-BATCH_START_TIME))
LOG_FILE_HANDLE.close()
print("\nBATCH SIMS COMPLETED in %ss" % str(BATCH_END_TIME-BATCH_START_TIME))


In [ ]:
# Used if python script crashes and leaves any files open

LOG_FILE_HANDLE.close()
netlist_file.close()
log_file.close()